In [ ]:
# NBVAL_SKIP
import os
os.environ['SPS_HOME'] = '/mnt/storage/annalena_data/sps_fsps'

# Load ssp template for FSPS

In [ ]:
# NBVAL_SKIP
from rubix.spectra.ssp.factory import get_ssp_template
ssp_fsps = get_ssp_template("FSPS")

In [ ]:
# NBVAL_SKIP
age_values = ssp_fsps.age
print(age_values.shape)

metallicity_values = ssp_fsps.metallicity
print(metallicity_values.shape)

In [ ]:
# NBVAL_SKIP
index = 80
metallicity_index = 3
initial_age_index = 100
learning = 0.5
tol = 1e-20

print(f"start age: {age_values[initial_age_index]}")
print(age_values[index], metallicity_values[metallicity_index])

# Configure pipeline

In [ ]:
# NBVAL_SKIP
from rubix.core.pipeline import RubixPipeline
import os
config = {
    "pipeline":{"name": "calc_gradient"},
    
    "logger": {
        "log_level": "DEBUG",
        "log_file_path": None,
        "format": "%(asctime)s - %(name)s - %(levelname)s - %(message)s",
    },
    "data": {
        "name": "IllustrisAPI",
        "args": {
            "api_key": os.environ.get("ILLUSTRIS_API_KEY"),
            "particle_type": ["stars"],
            "simulation": "TNG50-1",
            "snapshot": 99,
            "save_data_path": "data",
        },
        
        "load_galaxy_args": {
        "id": 14,
        "reuse": True,
        },
        
        "subset": {
            "use_subset": True,
            "subset_size": 2,
        },
    },
    "simulation": {
        "name": "IllustrisTNG",
        "args": {
            "path": "data/galaxy-id-14.hdf5",
        },
    
    },
    "output_path": "output",

    "telescope":
        {"name": "TESTGRADIENT",
         "psf": {"name": "gaussian", "size": 5, "sigma": 0.6},
         "lsf": {"sigma": 1.2},
         "noise": {"signal_to_noise": 1,"noise_distribution": "normal"},
         },
    "cosmology":
        {"name": "PLANCK15"},
        
    "galaxy":
        {"dist_z": 0.1,
         "rotation": {"type": "edge-on"},
        },
        
    "ssp": {
        "template": {
            "name": "FSPS",
        },
        "dust": {
                "extinction_model": "Cardelli89",
                "dust_to_gas_ratio": 0.01,
                "dust_to_metals_ratio": 0.4,
                "dust_grain_density": 3.5,
                "Rv": 3.1,
            },
    },        
}

In [ ]:
# NBVAL_SKIP
pipe = RubixPipeline(config)
inputdata = pipe.prepare_data()
rubixdata = pipe.run(inputdata)

# Set target values

In [ ]:
# NBVAL_SKIP
import jax.numpy as jnp

rubixdata.stars.age = jnp.array([age_values[index], age_values[index]])
rubixdata.stars.metallicity = jnp.array([metallicity_values[metallicity_index], metallicity_values[metallicity_index]])
rubixdata.stars.mass = jnp.array([[1.0, 1.0]])
rubixdata.stars.velocity = jnp.array([[0.0, 0.0, 0.0], [0.0, 0.0, 0.0]])

In [ ]:
# NBVAL_SKIP
target = pipe.run(rubixdata)

# Initial datacube

In [ ]:
# NBVAL_SKIP
rubixdata.stars.age = jnp.array([age_values[initial_age_index], age_values[initial_age_index]])
rubixdata.stars.metallicity = jnp.array([metallicity_values[metallicity_index], metallicity_values[metallicity_index]])
rubixdata.stars.mass = jnp.array([[1.0, 1.0]])
rubixdata.stars.velocity = jnp.array([[0.0, 0.0, 0.0], [0.0, 0.0, 0.0]])

In [ ]:
# NBVAL_SKIP
initial = pipe.run(rubixdata)

# Adam optimizer

In [ ]:
# NBVAL_SKIP
from rubix.pipeline import linear_pipeline as pipeline

pipeline_instance = RubixPipeline(config)

pipeline_instance._pipeline = pipeline.LinearTransformerPipeline(
    pipeline_instance.pipeline_config, 
    pipeline_instance._get_pipeline_functions()
)
pipeline_instance._pipeline.assemble()
pipeline_instance.func = pipeline_instance._pipeline.compile_expression()

In [ ]:
# NBVAL_SKIP
def loss_only_wrt_age(age, base_data, target):
    #data_modified = deepcopy(base_data)
    base_data.stars.age = age
    # Verwende den vor-kompilierten Funktionsaufruf
    output = pipeline_instance.func(base_data)
    loss = jnp.sum((output.stars.datacube - target.stars.datacube) ** 2)
    return loss

In [ ]:
# NBVAL_SKIP
import jax
import jax.numpy as jnp
import optax

#@jax.jit()
#def compute_loss_and_grad(loss_fn, age, data, target):
#    return jax.value_and_grad(loss_fn)(age, data, target)

def adam_optimization(loss_fn, age_init, data, target, learning_rate=0.1, tol=1e-3, max_iter=500):
    age = age_init  # initialize stellar age (a scalar)
    optimizer = optax.adam(learning_rate)
    optimizer_state = optimizer.init(age)
    
    age_history = []
    loss_history = []
    
    for i in range(max_iter):
        #loss, grad = compute_loss_and_grad(age, data, target)
        loss, grad = jax.value_and_grad(loss_fn)(age, data, target)
        #loss, grad = compute_loss_and_grad(loss_fn, age, data, target)
        loss_history.append(float(loss))
        age_history.append(float(age[0]))
        
        updates, optimizer_state = optimizer.update(grad, optimizer_state)
        age = optax.apply_updates(age, updates)
        
        # Optionally clip the age to enforce physical constraints, e.g., between 0 and 10.3
        age = jnp.clip(age, 0.0, 10.3)
        
        if jnp.linalg.norm(updates) < tol:
            print(f"Converged at iteration {i}")
            break

    return age, jnp.array(age_history), jnp.array(loss_history)

In [ ]:
# NBVAL_SKIP
data = initial  # Replace with your actual data if needed
target_value = target  # Replace with your actual target
age_init = jnp.array([age_values[initial_age_index], age_values[initial_age_index]])  # Initial guess for the stellar age

optimized_age, age_history, loss_history = adam_optimization(loss_only_wrt_age, age_init, data, target_value,
                                                               learning_rate=learning, tol=tol, max_iter=1000)

#optimized_age, age_history, loss_history = adam_scan(pipe.loss_only_wrt_age, age_init, data, target_value,
#                                                               learning_rate=learning, max_iter=500)

print(f"Optimized Age: {optimized_age}")



In [ ]:
# NBVAL_SKIP
import matplotlib.pyplot as plt
import numpy as np

# If loss_history is a JAX array, convert it to a NumPy array:
loss_history_np = np.array(loss_history)

# Create an array for the x-axis indices
indices = np.arange(len(loss_history_np))

plt.figure(figsize=(8, 6))
plt.plot(indices, loss_history_np, marker='o', linestyle='-')
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.title("Loss History")
plt.grid(True)
#plt.ylim(-1e-8, 2e-7)
#plt.savefig(f"./output/optimizer/optimization_progress_loss_metals{metallicity_index}_agestart{initial_age_index}_to{index}_learning{learning}_tol{tol}.png")
plt.show()

In [ ]:
# NBVAL_SKIP
import matplotlib.pyplot as plt
import numpy as np

# If age_history is a JAX array, convert it to a NumPy array:
age_history_np = np.array(age_history)

# Create an array for the x-axis indices
indices = np.arange(len(age_history_np))

plt.figure(figsize=(8, 6))
plt.plot(indices, age_history_np, marker='o', linestyle='-')
plt.hlines(y=age_values[index], xmin=0, xmax=len(age_history_np), color='r', linestyle='-')
plt.xlabel("Iteration")
plt.ylabel("Age")
plt.title("Age History")
plt.grid(True)
#plt.savefig(f"./output/optimizer/optimization_progress_age_metals{metallicity_index}_agestart{initial_age_index}_to{index}_learning{learning}_tol{tol}.png")
plt.show()



In [ ]:
# NBVAL_SKIP
#run the pipeline with the optimized age
#rubixdata.stars.age = optimized_age
i = 0
rubixdata.stars.age = jnp.array([age_history[i], age_history[i]])
rubixdata.stars.metallicity = jnp.array([metallicity_values[metallicity_index], metallicity_values[metallicity_index]])
rubixdata.stars.mass = jnp.array([[1.0, 1.0]])
rubixdata.stars.velocity = jnp.array([[0.0, 0.0, 0.0], [0.0, 0.0, 0.0]])

pipe = RubixPipeline(config)
rubixdata = pipe.run(rubixdata)

#plot the target and the optimized spectra
import matplotlib.pyplot as plt
wave = pipe.telescope.wave_seq

spectra_target = target.stars.datacube
spectra_optimitzed = rubixdata.stars.datacube

plt.plot(wave, spectra_target[0,0,:], label=f"Target age = {age_values[index]:.2f}")
plt.plot(wave, spectra_optimitzed[0,0,:], label=f"Optimized age = {age_history[i]:.2f}")
plt.xlabel("Wavelength [Å]")
plt.ylabel("Luminosity [L/Å]")
plt.title(f"Loss {loss_history[i]:.2e}")
plt.legend()
#plt.ylim(0, 0.0003)
plt.grid(True)
plt.show()

In [ ]:
# NBVAL_SKIP
#run the pipeline with the optimized age
#rubixdata.stars.age = optimized_age
i = -1
rubixdata.stars.age = jnp.array([age_history[i], age_history[i]])
rubixdata.stars.metallicity = jnp.array([metallicity_values[metallicity_index], metallicity_values[metallicity_index]])
rubixdata.stars.mass = jnp.array([[1.0, 1.0]])
rubixdata.stars.velocity = jnp.array([[0.0, 0.0, 0.0], [0.0, 0.0, 0.0]])

pipe = RubixPipeline(config)
rubixdata = pipe.run(rubixdata)

#plot the target and the optimized spectra
import matplotlib.pyplot as plt
wave = pipe.telescope.wave_seq

spectra_target = target.stars.datacube
spectra_optimitzed = rubixdata.stars.datacube

plt.plot(wave, spectra_target[0,0,:], label=f"Target age = {age_values[index]:.2f}")
plt.plot(wave, spectra_optimitzed[0,0,:], label=f"Optimized age = {age_history[i]:.2f}")
plt.xlabel("Wavelength [Å]")
plt.ylabel("Luminosity [L/Å]")
plt.title(f"Loss {loss_history[i]:.2e}")
plt.legend()
#plt.ylim(0, 0.0003)
plt.grid(True)
plt.show()